In [1]:
%matplotlib inline
# %matplotlib notebook

from __future__ import print_function

try:
    from importlib import reload
except:
    pass

import os
import sys
import warnings

import numpy as np
import pandas as pd
import astropy as ap
import sfdmap

import sqlite3
from sqlalchemy import create_engine

from matplotlib import pyplot as plt
import matplotlib.colors as mpl_colors 

import astropy.units as u

In [2]:
import pyCoCo as pccsims
import pycoco as pcc

reload(pcc)
reload(pcc.defaults)
reload(pcc.kcorr)
reload(pcc.classes)
reload(pccsims)

<module 'pyCoCo' from '/Users/berto/Code/CoCo/pyCoCo.cpython-36m-darwin.so'>

In [3]:
# %%timeit ## 1 loop, best of 3: 1min 31s per loop 
opsimdbpath = os.environ.get('OPSIMDBPATH')
print(opsimdbpath)
# opsimdbpath = "/Users/berto/data/LSST/OpSimOutputDBs/astro_lsst_01_1068_sqlite.db"
# opsimdbpath = "/Users/berto/data/LSST/OpSimOutputDBs/minion_1016_sqlite.db"

conn = create_engine('sqlite:///'+opsimdbpath, echo = False)
# opsimdf = pd.read_sql_table('Summary', con=conn)

/Users/berto/data/LSST/OpSimOutputDBs/minion_1016_sqlite.db


In [ ]:
opsimdf = pd.read_sql_query(sql = "SELECT * FROM Summary WHERE night > 365 AND night < 731", con = conn)

opsimdf.columns

In [ ]:
opsimdf.columns

Load in the Atmosphere

In [ ]:
# atmos = pcc.kcorr.load_atmosphere()
path = os.path.join(pcc._default_lsst_throughputs_path, "baseline/atmos_std.dat")
atmos = pcc.BaseFilterClass()
atmos.load(path, wavelength_u = u.nm, fmt = "ascii.commented_header", name = "atmosphere_12")
atmos.plot()

In [ ]:
# lsst_filters = ["LSST_u", "LSST_g", "LSST_r", "LSST_i", "LSST_z", "LSST_y"]
# lsst_filters = ["filter_u", "filter_g", "filter_r", "filter_i", "filter_z", " filter_y"]
lsst_filters = ["hardware_u", "hardware_g", "hardware_r", "hardware_i", "hardware_z", "hardware_y"]


fdict = {}

for filter_name in lsst_filters:
    filter_path = os.path.join(pcc._default_filter_dir_path, filter_name + ".dat")
    filter_path = os.path.join("/Users/berto/projects/LSST/throughputs/baseline/", filter_name + ".dat")
#     fdict[filter_name] = pcc.load_filter(filter_path)    
    fdict[filter_name] = pcc.FilterClass()
    fdict[filter_name].load(filter_path, wavelength_u = u.nm)
#     pcc.load_filter(filter_path)    

fig = plt.figure()

ax1 = fig.add_subplot(111)

for filter_name in lsst_filters:
    plt.plot(fdict[filter_name].data["wavelength"], fdict[filter_name].data["throughput"])
plt.show()


In [ ]:
hg = fdict['hardware_g']

In [ ]:
# atmos.wavelength
hg.wavelength.to(u.angstrom)

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(111)

for filter_name in lsst_filters:
    print("filter_name: ", filter_name )
    fdict[filter_name].resample_response(new_wavelength = atmos.wavelength)
    plt.plot(fdict[filter_name].wavelength, fdict[filter_name].throughput*atmos.throughput)
plt.show()


In [ ]:
# lsst_filters = ["LSST_u", "LSST_g", "LSST_r", "LSST_i", "LSST_z", "LSST_y"]
# lsst_filters = ["filter_u", "filter_g", "filter_r", "filter_i", "filter_z", " filter_y"]
lsst_filters = ["total_u", "total_g", "total_r", "total_i", "total_z", "total_y"]


fdict = {}

for filter_name in lsst_filters:
    filter_path = os.path.join(pcc._default_filter_dir_path, filter_name + ".dat")
    filter_path = os.path.join("/Users/berto/projects/LSST/throughputs/baseline/", filter_name + ".dat")
#     fdict[filter_name] = pcc.load_filter(filter_path)    
    fdict[filter_name] = pcc.FilterClass()
    fdict[filter_name].load(filter_path, wavelength_u = u.nm)
#     pcc.load_filter(filter_path)    

fig = plt.figure()

ax1 = fig.add_subplot(111)

for filter_name in lsst_filters:
    plt.plot(fdict[filter_name].data["wavelength"], fdict[filter_name].data["throughput"])
plt.show()


In [ ]:
for filter_key in fdict:
#     print(filter_key)
    fdict[filter_key].save(filename = filter_key.replace("total", "LSST") + ".dat", squash = True)

In [ ]:
pcc._default_filter_dir_path

In [ ]:
from astropy.table import Table

Table([fdict["total_g"].wavelength, fdict["total_g"].throughput], names = ["wavelength", "throughput"])

Load in the sky spectrum

In [ ]:
dark_sky_path = os.path.join(os.environ["LSST_THROUGHPUTS_BASELINE"],"darksky.dat")
darksky = pcc.SpectrumClass()
darksky.load(dark_sky_path, wavelength_u = u.nm, fmt = "ascii.commented_header", wmin = 3500*u.angstrom, wmax = 11000*u.angstrom,)
darksky.plot()

Which is equivalent to:

In [ ]:
darksky = pcc.kcorr.load_dark_sky_spectrum()
darksky.plot()

In [ ]:
lsst_filters = ["LSST_u", "LSST_g", "LSST_r", "LSST_i", "LSST_z", "LSST_y"]

for filter_name in lsst_filters:
    zp = pcc.kcorr.calc_AB_zp(filter_name)
    print(filter_name, " F_sky = ", pcc.kcorr.calc_spectrum_filter_flux(filter_name, darksky))
    print(filter_name, "m_sky = ", -2.5 * np.log10(pcc.kcorr.calc_spectrum_filter_flux(filter_name, darksky)) - zp)    

Create a df that just looks at the first observation in each filter

In [ ]:
opsimdf_f = opsimdf.drop_duplicates("filter")

opsimdf_f

In [ ]:
opsimdf_f[["filter", "filtSkyBrightness", "darkBright"]]

# OpSim m5 derivation 

Recreating the m5 values, as per http://smtn-002.lsst.io/en/latest/

\begin{split}m5 = C_m + dC_m + 0.50\,(m_{sky} - 21.0) + 2.5 log_{10}(0.7 /
FWHM_{eff}) \\
+ 1.25 log_{10}(expTime / 30.0) - k_{atm}\,(X-1.0)\end{split}

\begin{split}dC_m = dC_m^{inf} - 1.25 log_{10}(1 + (10^{(0.8\, dC_m^{inf} -
1)}/Tscale)\end{split}

\begin{split}Tscale = expTime / 30.0 * 10.0^{-0.4*(m_{sky} - m_{darksky})}\end{split}


In [ ]:
def calc_Tscale(df):
    Tscale = df["visitExpTime"].head()/30. * np.power(10., -0.4*())
    return Tscale


def calc_dCm():
    dCm = False
    return dCm


def calc_m5():
    m5 = False
    return m5

